# Demo for Charge Density-based Inverse Design

In [1]:
# Designate GPU to use
import os
gpu_ids = 0
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_ids}"

In [2]:
# import libraries
from termcolor import colored, cprint

import torch
import torch.backends.cudnn as cudnn
cudnn.benchmark = True

from models.base_model import create_model

import os

import warnings
warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

%load_ext autoreload
%autoreload 2

In [3]:
# Options for the model. please check `utils/demo_util.py` for more details
from utils.demo_util import ChargeDIFFOpt

seed = 42

opt = ChargeDIFFOpt(gpu_ids=gpu_ids, seed=seed)

opt.init_dset_args()
opt.init_model_args()

opt.batch_size = 32

device = opt.device

[*] CHGDIFF_TestOption initialized.


In [4]:
# Initialize MOFFUSION model

opt.model='chargediff_chemical_system'

opt.vq_cfg="./configs/vqvae.yaml"
opt.vq_ckpt="./saved_ckpt/vqvae.pth"

opt.df_cfg = './configs/chargediff_cond.yaml'
opt.ckpt = './saved_ckpt/chemsys.pth'

ChargeDIFF = create_model(opt)
cprint(f'[*] "{ChargeDIFF.name()}" loaded.', 'cyan')

[*] VQVAE: weight successfully load from: ./saved_ckpt/vqvae.pth
[*] weight successfully load from: ./saved_ckpt/chemsys.pth
[*] Model has been created: CHARGEDIFF-Model
[*] "CHARGEDIFF-Model" loaded.


In [5]:
from pymatgen.io.vasp.outputs import VolumetricData


# For Charge Manupulation
shape = torch.full((1,32,32,32), 0.0).to(device)

# For 1-dimensional Target
x_min, x_max = -1/3, 1/3
y_min, y_max = -1/3, 1/3
z_min, z_max = -1, 1

# For 2-dimensional Target
#x_min, x_max = -1, 1
#y_min, y_max = -1, 1
#z_min, z_max = -1/3, 1/3

xyz_dict = {'x': (x_min, x_max), 'y': (y_min, y_max), 'z': (z_min, z_max)}


# For Conditional Generation on Chemical System
chemical_system_condition = torch.zeros(17)

# Tm-O
chemical_system_condition[5] = 1.0
chemical_system_condition[13] = 1.0



batches = 1
batch_size = 10

for i in range(batches):
    
    gen_crys, chgden_tot = ChargeDIFF.cond_and_charge_manipulation(batch_size = batch_size, shape = shape, xyz_dict = xyz_dict, target=chemical_system_condition, dataset='mp_20_charge_PO4', cutoff = 0.5)


    for j in range(len(gen_crys)):

        cry = gen_crys[j]
        chgden = chgden_tot[j]
        
        structure = cry.structure
        
        volumetric = VolumetricData(structure = structure, data={'total': chgden})

        idx = i*batch_size + j

        structure.to_file(f'./sample/test/{idx}.cif')
        volumetric.write_file(f'./sample/test/{idx}.vasp')


100%|██████████| 1000/1000 [00:41<00:00, 23.82it/s]
